# PythonModel with type hints Quickstart
This notebook will demonstrates how to use type hints for data validation in MLflow PythonModel.

## Prerequisite

Install required packages: `pip install mlflow==2.20.0 openai==1.65.4`

Set your OpenAI API key with `os.environ["OPENAI_API_KEY"]="<YOUR_KEY>"`

## Create a model

In [1]:
import openai
import pydantic

import mlflow

# Use OpenAI model
messages = [{"role": "user", "content": "What is the MLflow?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
response

ChatCompletion(id='chatcmpl-B83LveI6Wc2RHEdbMwNGkFh8ZoehY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='MLflow is an open-source platform designed to manage the machine learning (ML) lifecycle, which includes components such as experimentation, reproducibility, and deployment. It provides a suite of tools to help data scientists and machine learning practitioners track experiments, package and share their code, and deploy models. Here are the key components of MLflow:\n\n1. **MLflow Tracking**: This component allows users to log and query experiments. You can track metrics, parameters, and artifacts (such as model files) associated with different runs of your machine learning models.\n\n2. **MLflow Projects**: This functionality enables users to package their data science code in a reusable and reproducible way. Projects are defined with a standard format that specifies their dependencies and how to run them.\n\n3. **MLflow Model

## Use MLflow PythonModel with type hints

In [2]:
# define your input schema
class Message(pydantic.BaseModel):
    role: str
    content: str


# inherit mlflow PythonModel
class MyModel(mlflow.pyfunc.PythonModel):
    # add type hint to model_input
    def predict(self, model_input: list[Message]) -> str:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=model_input)
        return response.choices[0].message.content


model = MyModel()
model.predict([{"role": "user", "content": "What is DSPy?"}])

'DSPy is a Python library designed for creating and managing decision systems, particularly in the context of data-driven applications. It provides tools for building models that can make decisions based on inputs from various data sources. The library aims to simplify the process of developing and deploying decision logic, which is often a complex task in machine learning and artificial intelligence projects.\n\nKey features of DSPy may include:\n\n1. **Declarative Syntax**: Allowing users to express decision logic in a clear and concise manner.\n2. **Integration with Data Sources**: Facilitating easy integration with various data workflows, making it simpler to utilize datasets for decision-making.\n3. **Evaluation and Testing**: Providing tools for evaluating and testing decision-making models, ensuring their accuracy and reliability.\n\nBy leveraging DSPy, data scientists and developers can focus on building effective decision systems without getting bogged down by the complexities

In [3]:
# An incorrect input will trigger validation error
model.predict(["What is DSPy?"])

MlflowException: Failed to validate data against type hint `list[Message]`, invalid elements: [('What is DSPy?', "Expecting example to be a dictionary or pydantic model instance for Pydantic type hint, got <class 'str'>")]

## Model logging

In [4]:
# log the model
with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(name="model", python_model=model, input_example=messages)

2025/03/06 19:07:07 INFO mlflow.models.signature: Inferring model signature from type hints
2025/03/06 19:07:07 INFO mlflow.models.signature: Running the predict function to generate output based on input example


In [5]:
model_info.signature

inputs: 
  [{content: string (required), role: string (required)} (required)]
outputs: 
  [string (required)]
params: 
  None

In [6]:
# load the pyfunc model
pyfunc_model = mlflow.pyfunc.load_model(model_info.model_uri)
# the same validation works for pyfunc model predict
pyfunc_model.predict(messages)

'MLflow is an open-source platform designed to manage the machine learning lifecycle. It provides tools for various stages of the machine learning process, including:\n\n1. **Experiment Tracking**: MLflow allows you to log and track experiments, enabling you to compare different runs and their performance metrics easily. You can log parameters, metrics, tags, and artifacts related to your models.\n\n2. **Projects**: MLflow Projects facilitate packaging and sharing code in a reusable format. This makes it easier to reproduce experiments and share your work with others.\n\n3. **Models**: MLflow Models provides a standard format for packaging machine learning models. It supports various flavors of models (e.g., TensorFlow, PyTorch, Scikit-learn) and allows you to deploy them to various environments (like Docker, cloud-based services, or local servers).\n\n4. **Registry**: The MLflow Model Registry provides a centralized repository to manage models, including versioning, annotation, and li

## Verify model before deployment

In [11]:
# verify model before serving
mlflow.models.predict(
    model_uri=model_info.model_uri,
    input_data=[{"role": "user", "content": "Where is New York?"}],
    env_manager="uv",
)

2025/03/06 19:10:16 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/03/06 19:10:16 INFO mlflow.utils.virtualenv: Creating a new environment in /var/folders/9g/psrbbvm92t712cy09d7_00d00000gp/T/tmpp18g94f9/envs/virtualenv_envs/mlflow-9d81fff15053e6e06e2edaefcc9e075d6c04a094 with python version 3.9.18 using uv
Using CPython 3.9.18 interpreter at: /Users/serena.ruan/miniconda3/envs/mlflow/bin/python3.9
Creating virtual environment at: /var/folders/9g/psrbbvm92t712cy09d7_00d00000gp/T/tmpp18g94f9/envs/virtualenv_envs/mlflow-9d81fff15053e6e06e2edaefcc9e075d6c04a094
Activate with: source /var/folders/9g/psrbbvm92t712cy09d7_00d00000gp/T/tmpp18g94f9/envs/virtualenv_envs/mlflow-9d81fff15053e6e06e2edaefcc9e075d6c04a094/bin/activate
2025/03/06 19:10:16 INFO mlflow.utils.virtualenv: Installing dependencies
Using Python 3.9.18 environment at: /var/folders/9g/psrbbvm92t712cy09d7_00d00000gp/T/tmpp18g94f9/envs/virtualenv_envs/mlflow-9d81fff15053e6e06e2edaef

{"predictions": "New York is a state located in the northeastern region of the United States. It is bordered by Vermont to the northeast, Massachusetts to the east, Connecticut to the southeast, and New Jersey and Pennsylvania to the south. The state also has access to the Atlantic Ocean to the southeast. The city of New York, often referred to simply as NYC, is the largest city in the state and is known for its significant cultural, financial, and historical influence."}

In [ ]:
model_info.model_uri

runs:/33b7da4d1693490b97934a5781964766/model


## Serve the model locally

In [ ]:
# run below command to serve the model locally
# mlflow models serve -m runs:/33b7da4d1693490b97934a5781964766/model -p 6666

In [19]:
import requests

from mlflow.models.utils import convert_input_example_to_serving_input

payload = convert_input_example_to_serving_input(
    [{"role": "user", "content": "Is British shorthair smart?"}]
)
resp = requests.post(
    "http://127.0.0.1:6666/invocations", data=payload, headers={"Content-Type": "application/json"}
)
resp.content

b'{"predictions": "British Shorthairs are generally considered to be intelligent cats, though their intelligence may manifest differently compared to some other breeds. They are known for their calm and laid-back demeanor, which can sometimes be mistaken for a lack of intelligence. In reality, they are capable of problem-solving and can be trained to perform basic commands or tricks, though they may not be as eager to please as some more active breeds.\\n\\nTheir intelligence is often reflected in their ability to adapt to their environment and their understanding of routines. British Shorthairs tend to be independent and may not seek out interaction as much as other, more playful breeds, but they can still form strong bonds with their owners. Essentially, while they might not be the most overtly intelligent cats, they possess a subtle understanding of their surroundings that reflects their adaptability and awareness."}'